In [2]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Adatok letöltése
# A különböző adatfájlok elérési útjainak meghatározása
splits = {
    'train': 'data/train-00000-of-00001-cc8461398e266567.parquet',
    'test': 'data/test-00000-of-00001-922aa10406034550.parquet',
    'eval': 'data/eval-00000-of-00001-dc793d916ae447cb.parquet'
}

# A tanító adatok betöltése a megadott elérési útvonalról
df = pd.read_parquet("hf://datasets/ckandemir/bitcoin_tweets_sentiment_kaggle/" + splits["train"])

# Mintavétel az adatokból (például 10%)
# Csak a teljes adatkészlet 10%-át használjuk a gyorsabb futás érdekében
df_sample = df.sample(frac=0.1, random_state=42)

# Az adatok első néhány sorának megtekintése
df_sample.head()

# Szövegek előkészítése és tisztítása
# A szövegek tisztítására szolgáló függvény definiálása
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # URL-ek eltávolítása
    text = re.sub(r'@\w+', '', text)  # Felhasználónevek eltávolítása
    text = re.sub(r'#\w+', '', text)  # Hashtagek eltávolítása
    text = re.sub(r'\d+', '', text)  # Számok eltávolítása
    text = text.lower()  # Kisbetűsítés
    return text

# A tisztító függvény alkalmazása az adatokra
df_sample['cleaned_text'] = df_sample['text'].apply(clean_text)

# Adatok szétválasztása tanító és teszt adatokra
# Az adatok 80%-át tanító, 20%-át teszt adatokra osztjuk
X_train, X_test, y_train, y_test = train_test_split(df_sample['cleaned_text'], df_sample['Sentiment'], test_size=0.2, random_state=42)

# Szövegek vektorizálása
# A TF-IDF vektorizáló inicializálása, amely legfeljebb 5000 jellemzőt használ
vectorizer = TfidfVectorizer(max_features=5000)
# A tanító adatok vektorizálása
X_train_vec = vectorizer.fit_transform(X_train)
# A teszt adatok vektorizálása
X_test_vec = vectorizer.transform(X_test)

# Több modell összeállítása
# Különböző gépi tanulási modellek definiálása
models = {
    'Naive Bayes': MultinomialNB(),
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier()
}

# Modellek kipróbálása és kiértékelése
# Minden modell betanítása és kiértékelése
for model_name, model in models.items():
    model.fit(X_train_vec, y_train)  # Modell betanítása
    y_pred = model.predict(X_test_vec)  # Előrejelzések készítése a teszt adatokra
    print(f"Model: {model_name}")  # Modell neve
    print(classification_report(y_test, y_pred))  # Kiértékelési jelentés

# Legjobb modell kiválasztása
# (Ez a rész függ a kiértékelés eredményétől, itt csak egy példa)
best_model = models['SVM']

# Működés bemutatása
# Függvény a szövegek osztályozására a legjobb modellel
def predict_sentiment(text):
    cleaned_text = clean_text(text)  # Szöveg tisztítása
    text_vec = vectorizer.transform([cleaned_text])  # Szöveg vektorizálása
    return best_model.predict(text_vec)  # Előrejelzés készítése

# Példa használat
example_text = "Bitcoin is going to the moon!"
print(f"Sentiment prediction for '{example_text}': {predict_sentiment(example_text)}")

Model: Naive Bayes
              precision    recall  f1-score   support

    Negative       0.49      0.33      0.39       752
    Positive       0.52      0.68      0.59       804

    accuracy                           0.51      1556
   macro avg       0.51      0.50      0.49      1556
weighted avg       0.51      0.51      0.49      1556

Model: SVM
              precision    recall  f1-score   support

    Negative       0.66      0.46      0.55       752
    Positive       0.61      0.78      0.68       804

    accuracy                           0.63      1556
   macro avg       0.64      0.62      0.61      1556
weighted avg       0.63      0.63      0.62      1556

Model: Random Forest
              precision    recall  f1-score   support

    Negative       0.62      0.53      0.57       752
    Positive       0.61      0.70      0.65       804

    accuracy                           0.62      1556
   macro avg       0.62      0.61      0.61      1556
weighted avg       0.62

In [3]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Ez ugyanaz a kód, csak egy pontosabb és jobb adathalmazzal, amit az órákon is használtunk

# Adatok letöltése
# Az IMDB adathalmaz betöltése
imdb_dataset = pd.read_csv("hf://datasets/scikit-learn/imdb/IMDB Dataset.csv")

# Az adatok első néhány sorának megtekintése
imdb_dataset.head()

# Mintavétel az adatokból (például 10%)
# Csak a teljes adatkészlet 10%-át használjuk a gyorsabb futás érdekében
imdb_sample = imdb_dataset.sample(frac=0.1, random_state=42)

# Szövegek előkészítése és tisztítása
# A szövegek tisztítására szolgáló függvény definiálása
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # URL-ek eltávolítása
    text = re.sub(r'@\w+', '', text)  # Felhasználónevek eltávolítása
    text = re.sub(r'#\w+', '', text)  # Hashtagek eltávolítása
    text = re.sub(r'\d+', '', text)  # Számok eltávolítása
    text = text.lower()  # Kisbetűsítés
    return text

# A tisztító függvény alkalmazása az adatokra
imdb_sample['cleaned_text'] = imdb_sample['review'].apply(clean_text)

# Adatok szétválasztása tanító és teszt adatokra
# Az adatok 80%-át tanító, 20%-át teszt adatokra osztjuk
X_train, X_test, y_train, y_test = train_test_split(imdb_sample['cleaned_text'], imdb_sample['sentiment'], test_size=0.2, random_state=42)

# Szövegek vektorizálása
# A TF-IDF vektorizáló inicializálása, amely legfeljebb 5000 jellemzőt használ
vectorizer = TfidfVectorizer(max_features=5000)
# A tanító adatok vektorizálása
X_train_vec = vectorizer.fit_transform(X_train)
# A teszt adatok vektorizálása
X_test_vec = vectorizer.transform(X_test)

# Több modell összeállítása
# Különböző gépi tanulási modellek definiálása
models = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=100)
}

# Modellek kipróbálása és kiértékelése
# Minden modell betanítása és kiértékelése
for model_name, model in models.items():
    model.fit(X_train_vec, y_train)  # Modell betanítása
    y_pred = model.predict(X_test_vec)  # Előrejelzések készítése a teszt adatokra
    print(f"Model: {model_name}")  # Modell neve
    print(classification_report(y_test, y_pred))  # Kiértékelési jelentés

# Legjobb modell kiválasztása
# (Ez a rész függ a kiértékelés eredményétől, itt csak egy példa)
best_model = models['Logistic Regression']

# Működés bemutatása
# Függvény a szövegek osztályozására a legjobb modellel
def predict_sentiment(text):
    cleaned_text = clean_text(text)  # Szöveg tisztítása
    text_vec = vectorizer.transform([cleaned_text])  # Szöveg vektorizálása
    return best_model.predict(text_vec)  # Előrejelzés készítése

# Példa használat
example_text = "This movie was fantastic!"
print(f"Sentiment prediction for '{example_text}': {predict_sentiment(example_text)}")

Model: Naive Bayes
              precision    recall  f1-score   support

    negative       0.83      0.84      0.84       506
    positive       0.84      0.83      0.83       494

    accuracy                           0.84      1000
   macro avg       0.84      0.84      0.84      1000
weighted avg       0.84      0.84      0.84      1000

Model: Logistic Regression
              precision    recall  f1-score   support

    negative       0.88      0.81      0.84       506
    positive       0.82      0.88      0.85       494

    accuracy                           0.85      1000
   macro avg       0.85      0.85      0.85      1000
weighted avg       0.85      0.85      0.85      1000

Sentiment prediction for 'This movie was fantastic!': ['positive']
